In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
TIME_STEPS = 28     # same as the height of the image
INPUT_SIZE = 28     # same as the width of the image
BATCH_SIZE = 50
BATCH_INDEX = 0
OUTPUT_SIZE = 10
CELL_SIZE = 50
LR = 0.001

In [3]:
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
# data pre-processing
X_train = X_train.reshape(-1, 28, 28) / 255.      # normalize 
X_test = X_test.reshape(-1, 28, 28) / 255.        # normalize
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [5]:
# build RNN model
model = Sequential()

In [6]:
# RNN cell
model.add(SimpleRNN(
    # for batch_input_shape, if using tensorflow as the backend, we have to put None for the batch_size.
    # Otherwise, model.evaluate() will get error.
    batch_input_shape=(None, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    output_dim=CELL_SIZE,
    unroll=True,
))

Instructions for updating:
Colocations handled automatically by placer.


/Users/magictavern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(batch_input_shape=(None, 28,..., unroll=True, units=50)`
  import sys


In [7]:
# output layer
model.add(Dense(OUTPUT_SIZE))
model.add(Activation('softmax'))

In [8]:
# optimizer
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# training
for step in range(4001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    cost = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX

    if step % 1000 == 0:
        cost, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], verbose=1)
        print('test cost: ', cost, 'test accuracy: ', accuracy)

10000/10000 [==============================] - 0s 12us/step
test cost:  0.21523398160934448 test accuracy:  0.9386000037193298
10000/10000 [==============================] - 0s 11us/step
test cost:  0.2310752421617508 test accuracy:  0.932699978351593
10000/10000 [==============================] - 0s 11us/step
test cost:  0.19421198964118958 test accuracy:  0.9455000162124634
10000/10000 [==============================] - 0s 11us/step
test cost:  0.2051953673362732 test accuracy:  0.942300021648407
10000/10000 [==============================] - 0s 11us/step
test cost:  0.1886356920003891 test accuracy:  0.948199987411499
